# **Food Delivery Time Prediction**

To predict the food delivery time in real-time, we need to calculate the distance between the food preparation point and the point of food consumption. After finding the distance between the restaurant and the delivery locations, we need to find relationships between the time taken by delivery partners to deliver the food in the past for the same distance.

So, for this task, we need a dataset containing data about the time taken by delivery partners to deliver food from the restaurant to the delivery location.

###importing the necessary Python libraries

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

###importing the necessary dataset

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/deliverytime.txt")
print(data.head())

In [ ]:
data.info()

look for whether this dataset contains any null values or not:

In [ ]:
data.isnull().sum()

The dataset does not have any null values.

# **Calculating Distance Between Two Latitudes and Longitudes**

The dataset doesn’t have any feature that shows the difference between the restaurant and the delivery location. All we have are the latitude and longitude points of the restaurant and the delivery location. We can use the **haversine** formula to calculate the distance between two locations based on their latitudes and longitudes.

In [ ]:
# Set the earth's radius (in kilometers)
R = 6371

# Convert degrees to radians
def deg_to_rad(degrees):
    return degrees * (np.pi/180)

# Function to calculate the distance between two points using the haversine formula
def distcalculate(lat1, lon1, lat2, lon2):
    d_lat = deg_to_rad(lat2-lat1)
    d_lon = deg_to_rad(lon2-lon1)
    a = np.sin(d_lat/2)**2 + np.cos(deg_to_rad(lat1)) * np.cos(deg_to_rad(lat2)) * np.sin(d_lon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c

# Calculate the distance between each pair of points
data['distance'] = np.nan

for i in range(len(data)):
    data.loc[i, 'distance'] = distcalculate(data.loc[i, 'Restaurant_latitude'],
                                        data.loc[i, 'Restaurant_longitude'],
                                        data.loc[i, 'Delivery_location_latitude'],
                                        data.loc[i, 'Delivery_location_longitude'])

We have now calculated the distance between the restaurant and the delivery location. We have also added a new feature in the dataset as distance. Let’s look at the dataset again:

In [ ]:
print(data.head())

In [ ]:
data.columns

#Data Exploration

Firstly we visualize the relationship between the distance and time taken to deliver the food

In [ ]:
figure = px.scatter(data_frame = data,
                    x="distance",
                    y="Time_taken(min)",
                    size="Time_taken(min)",
                    trendline="ols",
                    title = "Relationship Between Distance and Time Taken")
figure.show()

Now let’s have a look at the relationship between the time taken to deliver the food and the age of the delivery partner

In [ ]:
figure = px.scatter(data_frame = data,
                    x="Delivery_person_Age",
                    y="Time_taken(min)",
                    size="Time_taken(min)",
                    color = "distance",
                    trendline="ols",
                    title = "Relationship Between Time Taken and Age")
figure.show()

In [ ]:
figure = px.scatter(data_frame = data,
                    x="Delivery_person_Ratings",
                    y="Time_taken(min)",
                    size="Time_taken(min)",
                    color = "distance",
                    trendline="ols",
                    title = "Relationship Between Time Taken and Ratings")
figure.show()

In [ ]:
fig = px.box(data,
             x="Type_of_vehicle",
             y="Time_taken(min)",
             color="Type_of_order")
fig.show()

In [ ]:
#splitting data
from sklearn.model_selection import train_test_split
x = np.array(data[["Delivery_person_Age",
                   "Delivery_person_Ratings",
                   "distance"]])
y = np.array(data[["Time_taken(min)"]])
xtrain, xtest, ytrain, ytest = train_test_split(x, y,
                                                test_size=0.10,
                                                random_state=42)

# creating the LSTM neural network model
from keras.models import Sequential
from keras.layers import Dense, LSTM
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (xtrain.shape[1], 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.summary()

In [ ]:
# training the model
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(xtrain, ytrain, batch_size=1, epochs=9)

Epoch 1/9
41033/41033 [==============================] - 242s 6ms/step - loss: 69.4058
Epoch 2/9
41033/41033 [==============================] - 236s 6ms/step - loss: 63.9170
Epoch 3/9
22117/41033 [===============>..............] - ETA: 1:53 - loss: 61.0862

In [15]:
print("Food Delivery Time Prediction")
a = int(input("Age of Delivery Partner: "))
b = float(input("Ratings of Previous Deliveries: "))
c = int(input("Total Distance: "))

features = np.array([[a, b, c]])
print("Predicted Delivery Time in Minutes = ", model.predict(features))

Food Delivery Time Prediction
Age of Delivery Partner:  55
Ratings of Previous Deliveries: 4
Total Distance: 3
1/1 [==============================] - 2s 2s/step
Predicted Delivery Time in Minutes =  [[30.114202]]


In [17]:
!pip install fastapi
!pip install pydantic

In [31]:
import joblib

# Assuming you have a trained scikit-learn model saved as `model`
joblib.dump(model, '/content/drive/MyDrive/Colab Notebooks/assets/model.pkl')

['/content/drive/MyDrive/Colab Notebooks/assets/model.pkl']

In [32]:
import pickle

# Assuming you have a trained scikit-learn model saved as `model`
with open('/content/drive/MyDrive/Colab Notebooks/assets/model.pkl', 'wb') as f:
    pickle.dump(model, f)


In [33]:
import tensorflow as tf

# Assuming you have a trained TensorFlow model saved as `model`
model.save('/content/drive/MyDrive/Colab Notebooks/assets/model')  # The model will be saved as a directory


In [35]:
import torch

# Assuming you have a trained PyTorch model saved as `model`
torch.save(model, '/content/drive/MyDrive/Colab Notebooks/assets/model.pth')


In [37]:
from fastapi import FastAPI
from pydantic import BaseModel
import joblib

app = FastAPI()

# Load the trained machine learning model
model = joblib.load('/content/drive/MyDrive/Colab Notebooks/assets/model.pkl')

class DeliveryInput(BaseModel):
    age: int
    ratings: float
    distance: float

class DeliveryOutput(BaseModel):
    estimated_time: float

@app.post('/predict_delivery_time')
def predict_delivery_time(delivery_input: DeliveryInput):
    # Extract the input features from the request
    age = delivery_input.age
    ratings = delivery_input.ratings
    distance = delivery_input.distance

    # Perform any necessary preprocessing or transformations on the input features

    # Make the prediction using the loaded model
    prediction = model.predict([[age, ratings, distance]])

    # Prepare the response
    output = DeliveryOutput(estimated_time=prediction[0])

    return output


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.6 MB/s eta 0:00:00
